David Reed DS785 Spring 24

Summarization FLAN T5 xxl

 Set Runtime Type to T4 GPU for the lowest cost.

 Or to V100 High RAM

 Or to A100 High RAM

 but compute units are much more for the higher GPU, higher RAM options.



In [ ]:
! lscpu

Some problems with latest version of transformers so revert to version 4.31

In [ ]:
!pip install -q trl peft accelerate bitsandbytes datasets
!pip install -q xformers
!pip install -q git+https://github.com/huggingface/transformers@v4.31-release
#!pip install torch
!pip install -q evaluate
!pip install -q huggingface_hub huggingface
!pip install -q rouge_score





In [ ]:
import os
os.getcwd()

#!pip list

In [ ]:
import numpy as np
import pandas as pd
import torch
import tensorflow as tf
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    T5Tokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Trainer,
    )
from peft import (
    LoraConfig,
    PeftModel,
    AutoPeftModelForCausalLM,
    prepare_model_for_kbit_training,
    get_peft_model,
    )
from trl import SFTTrainer
import sklearn
import matplotlib
import importlib.util
import pathlib
import importlib_metadata
import tensorflow as tf
import evaluate
%load_ext tensorboard
import huggingface_hub
from datetime import datetime
import nltk
import csv


Check GPU and usage.

In [ ]:
device= 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:


gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

torch.cuda.memory_allocated()

In [ ]:
time=datetime.now().strftime("%m-%d-%H-%M-%S")
str_time = str(time)
str_time

In [ ]:
# use the flan xxl from hugging face
model_base =  "google/flan-t5-xxl"


# name for fine tuned model
model_fine_tuned = f"drFlanBase_{str_time}"

# dataset directory
dataset_dir = "reeddg/retail_products_chatgtp"

#output directiry
dir_out = f"reeddg/training_outputs/flan_sum_{str_time}"

runs_log_dir = f"./{dir_out}"



In [ ]:
print ("\n", model_base, "\n", model_fine_tuned,"\n", dataset_dir, "\n", dir_out,"\n", runs_log_dir)

When accessing Hugging Face you need to log in with Hugging Face cli or use the Secret Token.
This uses the HF_TOKEN and granting colab access, then it works through the rest of the code.

In [ ]:

# Load dataset from hugging face
dataset = load_dataset(dataset_dir)
# set review sentiment to class






# The code below is to create the Summary text to prepare for training.
---


In [ ]:
dft = dataset["train"].to_pandas()

In [ ]:
dft.describe()


In [ ]:
dft.head()

In [ ]:
dft["Description"][0]

In [ ]:
dft["Query"] = dft.apply(lambda x : x['Category'] + " " + x['Description'] + " " +  x['User Review'], axis=1)

#dft["text"] = dft[["Category", "Product Name", "User Review"]].apply(lambda x: "### Instruction:\n" + x["Category"] + " " + x["User Review"] + " \n\n### Response:\n" + x["Product Name"] + "\n", axis=1)
dft['Response'] = dft['Summary']

In [ ]:
print (f"{dft['Query'][0]}")

In [ ]:
dft[['Query','Description','Response']]

Set format for the prompt and answer that the model can use

For Flan T5 query

In [ ]:
dft["question"] = dft[["Query"]].apply(lambda x: "summarize: " + x["Query"] , axis=1)
dft["labels"] = dft["Response"]

df = dft[['question', 'labels']].copy()
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.3)

In [ ]:
dataset

------------------------------------------------------------------------------------------

In [ ]:
dataset["train"]

In [ ]:
dataset["train"][0]

In [ ]:
dataset["train"].to_pandas()

In [ ]:
#Tokenizer
#Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_base)

tokenizer.pad_token= tokenizer.eos_token


In [ ]:


tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["question"], truncation=True), batched=True)
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["labels"], truncation=True), batched=True)
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

In [ ]:
examples = dataset['train'][0]
inputs = examples["question"]
model_inputs = tokenizer(inputs, max_length=68, truncation=True, padding=True)
model_inputs

In [ ]:
def tokenize_function(examples):
    """Add prefix to the sentences, tokenize the text, and set the labels"""
    # The "inputs" are the tokenized answer:
    inputs = examples["question"]
    model_inputs = tokenizer(inputs, max_length=68, truncation=True, padding=True)

    # The "labels" are the tokenized outputs:
    labels = tokenizer(text_target=examples["labels"], max_length=32, truncation=True, padding=True)


    #labels["input_ids"] = [
    #    [(l if l != tokenizer.pad_token_id else 00) for l in label] for label in labels["input_ids"]
    #]

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Map the preprocessing function across the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['question'])

In [ ]:
tokenized_dataset['test'][0]['labels']

In [ ]:
    input_text = tokenizer.decode(tokenized_dataset['test'][0]['input_ids'], skip_special_tokens=True)
    output_text = np.where(tokenized_dataset['test']['labels']!= -100, tokenized_dataset['test'][0]['labels'], tokenizer.pad_token_id)
    output_text = tokenizer.decode(tokenized_dataset['test'][0]['labels'], skip_special_tokens=True)
    #label_text = tokenizer.decode(tokenized_dataset['test'][0]['question'], skip_special_tokens=True)

    print('Input:', input_text)
    print('Output:', output_text)
    #print('Label:', label_text)
    print('---')

In [ ]:
tokenized_dataset

In [ ]:
peft_config = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05, task_type="SEQ_2_SEQ_LM", bias="none"
    )



In [ ]:
peft_config

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)


# Load base moodel
model = AutoModelForSeq2SeqLM.from_pretrained(model_base, quantization_config=bnb_config,
    device_map={"": 0})



model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
model

## Create Evaluation

In [ ]:
# accuracy evaluation
rouge = evaluate.load("rouge")


In [ ]:
label_pad_token_id = tokenizer.eos_token_id
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    return_tensors ="pt"
)

LOAD in PEFT MODEL

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
# Run text generation pipeline with our model
#Input Prompt
prompt = "Detergent Tide Detergent has a fresh clean scent and it cuts through grease. Tide Detergent cleans well"
#Wrap the prompt using the right chat template

instruction = f"summarize: {prompt}"
print(instruction)
pipe = pipeline(task="summarization", model=model, tokenizer=tokenizer, max_length=32)
result = pipe(instruction)
#Trim the response, remove instruction manually
print(result[0]['summary_text'])

The Generation works despite the warning. Noted.

In [ ]:
training_arguments = Seq2SeqTrainingArguments(
        output_dir= runs_log_dir,
        learning_rate=2e-4,
        save_strategy="steps",
        logging_steps=10,
        num_train_epochs=10,
        generation_max_length=32,
        report_to="tensorboard",
        max_steps=-1,
        push_to_hub=True,
        eval_steps=1000,
    )

In [ ]:
dataset

In [ ]:
dataset['train'][0]

In [ ]:
type(dataset['train'])

In [ ]:

dataset['test'][0]


In [ ]:

trainer = Seq2SeqTrainer(
        model=model,
        args=training_arguments,
        tokenizer=tokenizer,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        #peft_config=peft_config,
        #dataset_text_field="text",
        #max_seq_length=68,
        data_collator=data_collator,
        #compute_metrics=compute_metrics,
    )

Now look at the GPU's, not required for evaluation, but to see GPU and CPU since colab has different options want to make sure GPU available.

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

gpus


visible_devices = tf.config.get_visible_devices()
for devices in visible_devices:
  print(devices)

tf.config.list_physical_devices('GPU')

In [ ]:
# Train model
trainer.train()

In [ ]:
logdir_path = f"{dir_out}/runs"
print (logdir_path)
f"{dir_out}"

In [ ]:
logdir_path = f"{dir_out}/runs"
print (logdir_path)
#huggingface_hub.upload_folder
#/content/reeddg/training_outputs3/runs/Mar24_14-57-33_ecd96771620a
%load_ext tensorboard
%tensorboard --logdir "$dir_out"


Create model on HuggingFace first

In [ ]:
# save fine-tuned model
ft_model_path = f"reeddg/test_train/sum_{model_fine_tuned}"
trainer.model.save_pretrained(ft_model_path)
trainer.save_model(ft_model_path)
trainer.push_to_hub(ft_model_path)


In [ ]:
# reload base model back in
base_model= AutoModelForSeq2SeqLM.from_pretrained(model_base,
    torch_dtype=torch.float16,
    device_map={"": 0})



In [ ]:
# merge base model and the fine-tuned model
merged_model= PeftModel.from_pretrained(base_model, ft_model_path)
merged_model= merged_model.merge_and_unload()

In [ ]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_base, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "right"

In [ ]:
# save merged model
merged_model_path= f"reeddg/test_model/mergedsum_{model_fine_tuned}"
merged_model.save_pretrained(merged_model_path)

In [ ]:
merged_model.push_to_hub("reeddg/T5sumxxl")
tokenizer.push_to_hub("reeddg/T5sumxxl")

In [ ]:
# Run text generation pipeline with our model
#Input Prompt
prompt = "Detergent Tide Detergent has a fresh clean scent and it cuts through grease. Tide Detergent cleans well"
#Wrap the prompt using the right chat template

instruction = f"summarize: {prompt}"
print(instruction)
#Using Pipeline from the hugging face
pipe = pipeline(task="summarization", model=merged_model, tokenizer=tokenizer, max_length=32)

result = pipe(instruction)
#Trim the response, remove instruction manually
#print(result[0]['summary_text'][len(instruction):].split('Response\n')[0])
print(result[0]['summary_text'])

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

torch.cuda.memory_allocated()


OUT OF GPU RAM, so restart and reload merged model from Hugging Face, Load the data, load the tokenizer and continue.

use this to load in the model later, after it has been stored to Hugging Face

In [ ]:
merged_model = AutoModelForSeq2SeqLM.from_pretrained("reeddg/T5sumxxl", quantization_config=bnb_config,
    device_map={"": 0})

In [ ]:
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_base,
                                                      device_map={"": 0},
                                                      quantization_config=bnb_config,
                                                      )

In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained("reeddg/T5sumxxl", use_fast=True)
#eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen(model,p, maxlen=68, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    #res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.1,num_beams=1,top_p=0.95,).to('cuda')
    res = model.generate(**toks.to("cuda"),max_new_tokens=maxlen,num_return_sequences=1,penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.5,repetition_penalty=1.2,).to('cuda')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)




In [ ]:
tokenized_dataset

In [ ]:
tokenized_dataset['test'][0]


In [ ]:
from time import perf_counter

qty_to_test=len(dataset['test'])
dialogues = dataset['test'][0:qty_to_test]['question']
human_baseline_summaries = dataset['test'][0:qty_to_test]['labels']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []
time_to_respond_orig = []
time_to_respond_peft = []

for idx, dialogue in enumerate(dialogues):
    human_baseline_text_output = human_baseline_summaries[idx]
    prompt = f"Instruct: Summarize the following conversation.\n{dialogue}\nOutput:\n"

    prompt = f"summarize: {dialogue}"
    print(prompt)

    start_time=perf_counter()

    original_model_res = gen(original_model,prompt,68)
    print(original_model_res)
    original_model_text_output = original_model_res[0]#.split('Response\n')[1]
    print(original_model_text_output)

    out_time = perf_counter() - start_time
    time_to_respond_orig.append(out_time)
    print(f"Time taken for original inference: {round(out_time,2)} seconds")

    start_time=perf_counter()

    peft_model_res = gen(merged_model,prompt,32)
    print(peft_model_res)
    peft_model_output = peft_model_res[0]#.split('Response\n')[1]
    print(peft_model_output)
    peft_model_text_output, success, result = peft_model_output.partition('#End')

    out_time = perf_counter() - start_time
    time_to_respond_peft.append(out_time)
    print(f"Time taken for peft inference: {round(out_time,2)} seconds")

    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))
zipped_time = list(zip(time_to_respond_orig, time_to_respond_peft))

df = pd.DataFrame(zipped_summaries, columns = ['baseline_summary', 'original_model_summary', 'peft_model_summary'])
df_time = pd.DataFrame(zipped_time, columns = ['orig_model_inference_time', 'peft_model_inference_time'])
df

In [ ]:
original_model_res[0]

In [ ]:
peft_model_res[0]


In [ ]:
[dialogues, human_baseline_summaries]

In [ ]:
import evaluate

rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

In [ ]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
peft_abs_imp_results=dict(zip(peft_model_results.keys(), improvement))
for key, value in peft_abs_imp_results.items():
    print(f'{key}: {value*100:.2f}%')

In [ ]:
print (f"Inference time peft: {round(np.average(time_to_respond_peft),4)}")
print (f"Inference time original: {round(np.average(time_to_respond_orig),4)}")

time_p2o = np.subtract(time_to_respond_peft, time_to_respond_orig)
time_2inf = np.average(time_p2o)
time_p=["Time2Imp"]
#print(time_p)
time_p = [*time_p, *time_p2o]
#print (time_p)


if(time_2inf>0):
  print(f"Inference time for original model is on average {round(time_2inf,2)} seconds better than the peft model.")
else:
  print(f"Inference time for peft model is on average {round(-time_2inf,2)} seconds better than the original model.")

In [ ]:

def ret_aray(r,rge_dict):
   outputs=[r]
   for k,v in rge_dict.items():
    #print(k,v)
    outputs.append(v)
   return(outputs)

def ret_arayk(r,rge_dict):
   outputs=[r]
   for k,v in rge_dict.items():
    #print(k,v)
    outputs.append(k)
   return(outputs)


In [ ]:
file= model_fine_tuned + "summary.csv"

with open(file, 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(ret_arayk("col", original_model_results))
  print(original_model_results)
  writer.writerow(ret_aray("orig",original_model_results))
  print(peft_model_results)
  writer.writerow(ret_aray("peft",peft_model_results))
  print(peft_abs_imp_results)
  writer.writerow(ret_aray("peft-orig",peft_abs_imp_results))
  print(time_p)
  writer.writerow(time_p)
file.close()



file_time= "time_" + model_fine_tuned + "summary.csv"

df_time.to_csv(file_time,encoding='utf-8')



In [ ]:
print(model_fine_tuned)
